## Part 1 & 2
> - Decode the structure of your hexadecimal-encoded BITS transmission; what do you get if you add up the version numbers in all packets?
> - What do you get if you evaluate the expression represented by your hexadecimal-encoded BITS transmission?

In [25]:
from pydantic import BaseModel, Field
from typing import Optional
from math import prod
from pydantic import BaseModel, Field
from typing import Optional
from math import prod

# Parse input to binary string
inp = "".join([bin(int(x,16))[2:].zfill(4) for x in list(open("input.txt").read())])

# Helper data-class just for fun
class Packet(BaseModel):    
    version: int = Field(...)
    type_id: int = Field(...)
    payload: int = Field(None)
    length_type_id: Optional[int] = Field(None)
    subpacket_len: Optional[int] = Field(None)
    subpacket_cnt: Optional[int] = Field(None)
         
# Type-ID to python function mapper
ops = { 0: "sum", 1: "prod", 2: "min", 3: "max", 5: "gt", 6: "lt", 7: "eq" }

# Missing/custom functions
def gt(inp:list) -> int:
    return 1 if inp[0] > inp[1] else 0
def lt(inp:list) -> int:
    return 1 if inp[0] < inp[1] else 0
def eq(inp:list) -> int:
    return 1 if inp[0] == inp[1] else 0

# Recursive input parsing
def parse(inp: str, is_sub:bool = False, sub_cnt:int = -1) -> tuple[int,str,list[Packet]]:    
    ii, packets, equation = 6, [], ""
    p = {
        "version": int(inp[:3], 2),
        "type_id": int(inp[3:6], 2)
    }
    if p["type_id"] == 4: # Literal packet
        num = ""
        while True:
            grp = inp[ii:ii+5]
            ii += 5
            num += grp[1:]
            if grp[0] == "0":
                break
        p["payload"] = int(num,2)
        equation += f"{p['payload']},"
    else:                 # Operator packet
        equation += f"{ops[p['type_id']]}(["
        p["length_type_id"] = int(inp[6:7], 2)
        if p["length_type_id"] == 0: # 15 bit mode
            p["subpacket_len"] = int(inp[7:22], 2)
            ii_n, equa, pp = parse(inp[22:22+p["subpacket_len"]], True)
            equation += equa
            ii = ii_n + 22
            packets = packets + pp           
        else:                        # 11 bit mode
            p["subpacket_cnt"] = int(inp[7:18], 2)
            ii_n, equa, pp = parse(inp[18:], True, sub_cnt=p["subpacket_cnt"]-1)
            equation += equa
            ii = ii_n + 18 
            packets = packets + pp
        equation = equation.rstrip(",") + "]),"
    if is_sub and ii < len(inp) and sub_cnt == -1:
        ii_n, equa, pp = parse(inp[ii:], True)
        equation += equa
        ii += ii_n
        packets = packets + pp
    elif is_sub and sub_cnt > 0:
        ii_n, equa, pp = parse(inp[ii:], True, sub_cnt-1)
        equation += equa
        ii += ii_n
        packets = packets + pp
    return (ii,equation.rstrip(","),[Packet(**p)]+packets)

# Parse input and print results
_, equation, packets = parse(inp)
print(f"Sum of all version numbers: {sum((x.version for x in packets))}\n")
print(f"Equation: {equation}\n")
print(f"Result of equation: {eval(equation)}")

Sum of all version numbers: 886

Equation: sum([prod([425542,lt([247,247])]),sum([121,21236]),prod([gt([sum([11,12,11]),sum([7,10,7])]),32566]),prod([lt([sum([8,7,15]),sum([6,11,10])]),4507180]),min([prod([prod([sum([prod([max([prod([min([prod([min([prod([max([sum([sum([max([sum([sum([min([prod([prod([130])])])])])])])])])])])])])])])])])])])]),139930778832,prod([gt([52,667118]),10]),602147,max([62199]),prod([14849899,lt([11716,26963])]),prod([4083,gt([135,135])]),prod([135,217,224]),73,prod([sum([13,4,9]),sum([12,15,7]),sum([13,10,9])]),min([194]),prod([182,197,136,2,242]),prod([226,142,34,124]),max([4025,186042]),min([30059,126119002]),min([9,260,162]),prod([lt([4,4]),28699]),prod([1945,eq([1714,1714])]),prod([7,lt([1545,108])]),sum([12]),prod([200,gt([31050,655605])]),3154,prod([3,lt([64896,116])]),3055,prod([13]),min([48082,226938,1175,68077774919]),sum([66,15,181,1380642642,11831587]),prod([241,59]),prod([150,gt([2742,113])]),37007908601,max([52444,11,13008816,2935]),20723,8,prod(